In [14]:
# set SLM

from meadowlark import Meadowlark
import numpy as np

slm = Meadowlark(
    verbose=True,
    sdk_path="C:\\Program Files\\Meadowlark Optics\\Blink OverDrive Plus",
    # lut_path="C:\\Program Files\\Meadowlark Optics\\SDK\\slm5691_at635.LUT",
    lut_path="C:\\Program Files\\Meadowlark Optics\\SDK\\1920x1152_linearVoltage.LUT",
    wav_um=0.550,
    pitch_um=(9.2, 9.2),
)

print(slm.shape)


def generate_and_load_fresnel_lens(slm, focal_length_mm, angle_x_mrad, angle_y_mrad, 
                                   wavelength=550e-9, pixel_size=9.2e-6, two_pi_value=190):
    """
    Generate and load a Fresnel lens phase pattern with beam steering to SLM.
    
    Parameters:
    -----------
    slm : object
        The SLM object with .shape attribute and .set_phase() method
    focal_length_mm : float
        Focal length in millimeters (positive for converging, negative for diverging)
    angle_x_mrad : float
        Steering angle in x direction in milliradians
    angle_y_mrad : float
        Steering angle in y direction in milliradians
    wavelength : float
        Wavelength of light in meters (default: 550nm)
    pixel_size : float
        Size of the SLM pixel in meters (default: 9.2μm)
    two_pi_value : int
        SLM grayscale value corresponding to 2π phase shift (default: 190)
    
    Returns:
    --------
    phase_uint8 : ndarray
        The phase pattern as uint8 array that was loaded to the SLM
    """
    # Get SLM shape
    height, width = slm.shape
    
    # Convert focal length to meters
    focal_length_m = focal_length_mm * 1e-3
    
    # Create coordinate grid
    y, x = np.indices((height, width))
    
    # Calculate center of the SLM
    center_x = width / 2
    center_y = height / 2
    
    # Calculate distances from center in meters
    x_meters = (x - center_x) * pixel_size
    y_meters = (y - center_y) * pixel_size
    r_squared = x_meters**2 + y_meters**2
    
    # Calculate Fresnel lens phase
    # φ(x,y) = (2π/λ) * (f - sqrt(f² + x² + y²))
    f_squared = focal_length_m**2
    lens_phase = (2 * np.pi / wavelength) * (focal_length_m - np.sqrt(f_squared + r_squared))
    
    # Calculate linear phase for beam steering
    # Convert milliradians to radians
    angle_x_rad = angle_x_mrad * 1e-3
    angle_y_rad = angle_y_mrad * 1e-3
    
    # Phase gradients for steering
    phase_gradient_x = 2 * np.pi * np.sin(angle_x_rad) / wavelength
    phase_gradient_y = 2 * np.pi * np.sin(angle_y_rad) / wavelength
    
    # Linear phase for beam steering
    steering_phase = phase_gradient_x * x_meters + phase_gradient_y * y_meters
    
    # Combine lens and steering phases
    total_phase = lens_phase + steering_phase
    
    # Wrap phase to [0, 2π) range
    total_phase = total_phase % (2 * np.pi)
    
    # Convert to uint8 for SLM
    phase_uint8 = np.uint8(total_phase / (2 * np.pi) * two_pi_value)
    
    # Load to SLM
    slm.set_phase(phase_uint8)
    
    return phase_uint8




Validating DPI awareness...success
Loading Blink SDK libraries...success
Initializing SDK...success
Found 1 SLM controller(s)
Loading LUT file...success
(1152, 1920)


In [16]:
# Example usage:
phase = generate_and_load_fresnel_lens(slm, 
                                       focal_length_mm=238, 
                                       angle_x_mrad=5.0, 
                                       angle_y_mrad=-3.0)

In [15]:
import cv2
import time
import numpy as np

# 加载预训练的人脸检测器
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 打开摄像头
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# 相机参数 (根据你的相机调整这些值)
# 大多数网络摄像头的FOV约为60-75度（水平）和45-60度（垂直）
FOV_HORIZONTAL = 15  # 水平视场角（度）
FOV_VERTICAL = 10    # 垂直视场角（度）

# 获取帧尺寸
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 计算图像中心
center_x = frame_width // 2
center_y = frame_height // 2

# FPS计算
fps_time = 0
fps_counter = 0
fps_display = 0

def calculate_angular_position(face_center_x, face_center_y, img_width, img_height, fov_h, fov_v):
    """
    计算人脸中心相对于图像中心的角度位置
    
    参数:
    - face_center_x, face_center_y: 人脸中心坐标
    - img_width, img_height: 图像尺寸
    - fov_h, fov_v: 水平和垂直视场角（度）
    
    返回:
    - theta_x: 水平角度（正值表示右侧，负值表示左侧）
    - theta_y: 垂直角度（正值表示上方，负值表示下方）
    """
    # 图像中心
    img_center_x = img_width / 2
    img_center_y = img_height / 2
    
    # 计算相对位置（归一化到-1到1）
    rel_x = (face_center_x - img_center_x) / (img_width / 2)
    rel_y = -(face_center_y - img_center_y) / (img_height / 2)  # Y轴反转
    
    # 计算角度
    theta_x = rel_x * (fov_h / 2)
    theta_y = rel_y * (fov_v / 2)
    
    return theta_x, theta_y

def draw_crosshair(img, x, y, size=20, color=(0, 255, 255), thickness=2):
    """绘制十字准心"""
    cv2.line(img, (x - size, y), (x + size, y), color, thickness)
    cv2.line(img, (x, y - size), (x, y + size), color, thickness)

print("按 'q' 退出程序")
print(f"相机FOV设置: {FOV_HORIZONTAL}° (水平) x {FOV_VERTICAL}° (垂直)")
print(f"分辨率: {frame_width} x {frame_height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 转换为灰度图（Haar Cascade在灰度图上工作）
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 检测人脸
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=10,
        minSize=(80, 80)
    )
    
    # 绘制图像中心十字准心
    draw_crosshair(frame, center_x, center_y, size=30, color=(128, 128, 128), thickness=1)
    
    # 处理每个检测到的人脸
    for i, (x, y, w, h) in enumerate(faces):
        # 绘制人脸边界框
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # 计算人脸中心
        face_center_x = x + w // 2
        face_center_y = y + h // 2
        
        # 绘制人脸中心点
        cv2.circle(frame, (face_center_x, face_center_y), 5, (0, 0, 255), -1)
        
        # 计算角度位置
        theta_x, theta_y = calculate_angular_position(
            face_center_x, face_center_y,
            frame_width, frame_height,
            FOV_HORIZONTAL, FOV_VERTICAL
        )
        
        # 计算相对位置（百分比）
        rel_x_percent = ((face_center_x - center_x) / center_x) * 100
        rel_y_percent = -((face_center_y - center_y) / center_y) * 100  # Y轴反转
        
        # 显示人脸信息
        info_y = y - 10 if y > 100 else y + h + 20
        
        # 角度信息
        angle_text = f"Angle: ({theta_x:.1f}, {theta_y:.1f})"
        cv2.putText(frame, angle_text, (x, info_y),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        
        # 相对位置信息
        pos_text = f"Pos: ({rel_x_percent:.0f}%, {rel_y_percent:.0f}%)"
        cv2.putText(frame, pos_text, (x, info_y + 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        
        # 绘制从图像中心到人脸中心的线
        cv2.line(frame, (center_x, center_y), (face_center_x, face_center_y), 
                (0, 255, 255), 1)
    
    # 计算FPS
    fps_counter += 1
    if time.time() - fps_time > 1:
        fps_display = fps_counter
        fps_counter = 0
        fps_time = time.time()
    
    # 显示信息面板
    panel_y = 30
    cv2.putText(frame, f'FPS: {fps_display}', (10, panel_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    cv2.putText(frame, f'Faces: {len(faces)}', (10, panel_y + 25),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
    
    cv2.putText(frame, f'FOV: {FOV_HORIZONTAL}x{FOV_VERTICAL} deg', (10, panel_y + 50),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
    
    # 绘制坐标轴说明（右上角）
    legend_x = frame_width - 150
    legend_y = 30
    cv2.putText(frame, 'Center: (0, 0)', (legend_x, legend_y),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    cv2.putText(frame, '+X: Right', (legend_x, legend_y + 20),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    cv2.putText(frame, '+Y: Up', (legend_x, legend_y + 40),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    
    # 显示结果
    cv2.imshow('Face Tracking with Angular Position', frame)
    
    # 按键处理
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('h'):  # 增加水平FOV
        FOV_HORIZONTAL = min(FOV_HORIZONTAL + 5, 180)
        print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('g'):  # 减少水平FOV
        FOV_HORIZONTAL = max(FOV_HORIZONTAL - 5, 10)
        print(f"FOV水平: {FOV_HORIZONTAL}°")
    elif key == ord('v'):  # 增加垂直FOV
        FOV_VERTICAL = min(FOV_VERTICAL + 5, 180)
        print(f"FOV垂直: {FOV_VERTICAL}°")
    elif key == ord('b'):  # 减少垂直FOV
        FOV_VERTICAL = max(FOV_VERTICAL - 5, 10)
        print(f"FOV垂直: {FOV_VERTICAL}°")

print("\n程序结束")
print("按键说明:")
print("  q - 退出")
print("  h/g - 增加/减少水平FOV")
print("  v/b - 增加/减少垂直FOV")

cap.release()
cv2.destroyAllWindows()

按 'q' 退出程序
相机FOV设置: 15° (水平) x 10° (垂直)
分辨率: 1024 x 576

程序结束
按键说明:
  q - 退出
  h/g - 增加/减少水平FOV
  v/b - 增加/减少垂直FOV
